# Tuning and Optimizing Neural Networks - Lab

## Introduction

Now that you've practiced regularization, initialization, and optimization techniques, its time to synthesize these concepts into a cohesive modeling pipeline.  

With this pipeline, you will not only fit an initial model but also attempt to improve it. Your final model selection will pertain to the test metrics across these models. This will more naturally simulate a problem you might be faced with in practice, and the various modeling decisions you are apt to encounter along the way.  

Recall that our end objective is to achieve a balance between overfitting and underfitting. You've seen the bias variance trade-off, and the role of regularization in order to reduce overfitting on training data and improving generalization to new cases. Common frameworks for such a procedure include train/validate/test methodology when data is plentiful, and K-folds cross-validation for smaller, more limited datasets. In this lab, you'll perform the latter, as the dataset in question is fairly limited. 

## Objectives

You will be able to:

* Apply normalization as a preprocessing technique 
* Implement a K-folds cross validation modeling pipeline for deep learning models 
* Apply regularization techniques to improve your model's performance 

## Load the data

First, run the following cell to import all the neccessary libraries and classes you will need in this lab. 

In [49]:
# Necessary libraries and classes
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import cross_val_predict
from keras import models
from keras import layers
from keras import regularizers
from scikeras.wrappers import KerasRegressor

In this lab you'll be working with the *The Lending Club* data. 

- Import the data available in the file `'loan_final.csv'` 
- Drop rows with missing values in the `'total_pymnt'` column (this is your target column) 
- Print the first five rows of the data 
- Print the dimensions of the data 

In [50]:
# Import the data
data = pd.read_csv("loan_final.csv")
data.head()

,loan_amnt,funded_amnt_inv,term,int_rate,installment,grade,emp_length,home_ownership,annual_inc,verification_status,loan_status,purpose,addr_state,total_acc,total_pymnt,application_type
0,5000.0,4975.0,36 months,10.65%,162.87,B,10+ years,RENT,24000.0,Verified,Fully Paid,credit_card,AZ,9.0,5863.155187,Individual
1,2500.0,2500.0,60 months,15.27%,59.83,C,< 1 year,RENT,30000.0,Source Verified,Charged Off,car,GA,4.0,1014.530000,Individual
2,2400.0,2400.0,36 months,15.96%,84.33,C,10+ years,RENT,12252.0,Not Verified,Fully Paid,small_business,IL,10.0,3005.666844,Individual
3,10000.0,10000.0,36 months,13.49%,339.31,C,10+ years,RENT,49200.0,Source Verified,Fully Paid,other,CA,37.0,12231.890000,Individual
4,3000.0,3000.0,60 months,12.69%,67.79,B,1 year,RENT,80000.0,Source Verified,Fully Paid,other,OR,38.0,4066.908161,Individual


In [51]:
# Drop rows with no target value
data = data.dropna(subset=["total_pymnt"])

# Print the first five rows
data.head()

,loan_amnt,funded_amnt_inv,term,int_rate,installment,grade,emp_length,home_ownership,annual_inc,verification_status,loan_status,purpose,addr_state,total_acc,total_pymnt,application_type
0,5000.0,4975.0,36 months,10.65%,162.87,B,10+ years,RENT,24000.0,Verified,Fully Paid,credit_card,AZ,9.0,5863.155187,Individual
1,2500.0,2500.0,60 months,15.27%,59.83,C,< 1 year,RENT,30000.0,Source Verified,Charged Off,car,GA,4.0,1014.530000,Individual
2,2400.0,2400.0,36 months,15.96%,84.33,C,10+ years,RENT,12252.0,Not Verified,Fully Paid,small_business,IL,10.0,3005.666844,Individual
3,10000.0,10000.0,36 months,13.49%,339.31,C,10+ years,RENT,49200.0,Source Verified,Fully Paid,other,CA,37.0,12231.890000,Individual
4,3000.0,3000.0,60 months,12.69%,67.79,B,1 year,RENT,80000.0,Source Verified,Fully Paid,other,OR,38.0,4066.908161,Individual


In [52]:
# Print the dimensions of data 
data.shape

(42535, 16)

## Generating a Hold Out Test Set

While we will be using K-fold cross validation to select an optimal model, we still want a final hold out test set that is completely independent of any modeling decisions. As such, pull out a sample of 30% of the total available data. For consistency of results, use random seed 42. 

In [53]:
# Features to build the model
features = ['loan_amnt', 'funded_amnt_inv', 'installment', 'annual_inc', 
            'home_ownership', 'verification_status', 'emp_length']

X = data[features]
y = data[['total_pymnt']]

# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

## Preprocessing (Numerical features) 

- Fill all missing values in numeric features with their respective means 
- Standardize all the numeric features  
- Convert the final results into DataFrames 

In [54]:
# Select continuous features
cont_features = ['loan_amnt', 'funded_amnt_inv', 'installment', 'annual_inc']

X_train_cont = X_train.loc[:, cont_features]
X_test_cont = X_test.loc[:, cont_features]

# Instantiate SimpleImputer - fill the missing values with the mean
si = SimpleImputer()
si.set_output(transform="pandas")

# Fit and transform the training data
X_train_imputed = si.fit_transform(X_train_cont)

# Transform test data
X_test_imputed = si.transform(X_test_cont)

# Instantiate StandardScaler
ss_X = StandardScaler()
ss_X.set_output(transform="pandas")

# Fit and transform the training data
X_train_scaled = ss_X.fit_transform(X_train_imputed)

# Transform test data
X_test_scaled = ss_X.transform(X_test_imputed)

## Preprocessing (Categorical features) 

- Fill all missing values in categorical features with the string `'missing'` 
- One-hot encode all categorical features 
- Convert the final results into DataFrames 


In [55]:
# Select only the categorical features
cat_features = ['home_ownership', 'verification_status', 'emp_length']
X_train_cat = X_train.loc[:, cat_features]
X_test_cat = X_test.loc[:, cat_features]

# Fill missing values with the string 'missing'
X_train_cat = X_train_cat.fillna("missing")
X_test_cat = X_test_cat.fillna("missing")

# OneHotEncode categorical variables
ohe = OneHotEncoder(sparse_output=False)
ohe.set_output(transform="pandas")

# Transform training and test sets
X_train_ohe = ohe.fit_transform(X_train_cat)
X_test_ohe = ohe.transform(X_test_cat)

# Get all categorical feature names
# cat_columns = ohe.get_feature_names_out(input_features=X_train_ohe.columns)

# Fit and transform the training data
X_train_categorical = X_train_ohe

# Transform test data
X_test_categorical = X_test_ohe

X_train_categorical.columns

Index(['home_ownership_MORTGAGE', 'home_ownership_NONE',
       'home_ownership_OTHER', 'home_ownership_OWN', 'home_ownership_RENT',
       'verification_status_Not Verified',
       'verification_status_Source Verified', 'verification_status_Verified',
       'emp_length_1 year', 'emp_length_10+ years', 'emp_length_2 years',
       'emp_length_3 years', 'emp_length_4 years', 'emp_length_5 years',
       'emp_length_6 years', 'emp_length_7 years', 'emp_length_8 years',
       'emp_length_9 years', 'emp_length_< 1 year', 'emp_length_missing'],
      dtype='object')

Run the below cell to combine the numeric and categorical features. 

In [56]:
# Combine continuous and categorical feature DataFrames
X_train_all = pd.concat([X_train_scaled, X_train_categorical], axis=1)
X_test_all = pd.concat([X_test_scaled, X_test_categorical], axis=1)

# Number of input features
n_features = X_train_all.shape[1]

- Standardize the target DataFrames (`y_train` and `y_test`) 

In [57]:
# Instantiate StandardScaler
ss_y = StandardScaler()
ss_y.set_output(transform="pandas")

# Fit and transform Y (train)
y_train_scaled = ss_y.fit_transform(y_train)

# Transform test Y (test)
y_test_scaled = ss_y.transform(y_test)

## Define a K-fold Cross Validation Methodology

Now that your have a complete holdout test set, you will perform k-fold cross-validation using the following steps: 

- Create a function that returns a compiled deep learning model 
- Use the wrapper function `KerasRegressor()` that defines how these folds are trained 
- Call the `cross_val_predict()` function to perform k-fold cross-validation 

In the cell below, we've defined a baseline model that returns a compiled Keras models. 

In [58]:
# Define a function that returns a compiled Keras model 
def create_baseline_model():
    
    # Initialize model
    model = models.Sequential()

    # First hidden layer
    model.add(layers.Input((n_features,)))
    model.add(layers.Dense(10, activation='relu'))

    # Second hidden layer
    model.add(layers.Dense(5, activation='relu'))

    # Output layer
    model.add(layers.Dense(1, activation='linear'))

    # Compile the model
    model.compile(optimizer='SGD', 
                  loss='mse',  
                  metrics=['mse']) 
    
    # Return the compiled model
    return model

Wrap `create_baseline_model` inside a call to `KerasRegressor()`, and: 

- Train for 150 epochs 
- Set the batch size to 256 

> NOTE: Refer to the [documentation](https://keras.io/scikit-learn-api/) to learn about `KerasRegressor()`.  

In [59]:
# Wrap the above function for use in cross-validation
keras_wrapper_1 = KerasRegressor(
    create_baseline_model,
    epochs=150,
    batch_size=256
)

Use `cross_val_predict()` to generate cross-validated predictions with: 
- 5-fold cv 
- scaled input (`X_train_all`) and output (`y_train_scaled`) 

In [60]:
# ⏰ This cell may take several mintes to run
# Generate cross-validated predictions
np.random.seed(123)
cv_baseline_preds = cross_val_predict(keras_wrapper_1, X_train_all, y_train_scaled)

Epoch 1/150
94/94 ━━━━━━━━━━━━━━━━━━━━ 0s 653us/step - loss: 0.8022 - mse: 0.8022
Epoch 2/150
94/94 ━━━━━━━━━━━━━━━━━━━━ 0s 579us/step - loss: 0.3013 - mse: 0.3013
Epoch 3/150
94/94 ━━━━━━━━━━━━━━━━━━━━ 0s 552us/step - loss: 0.2518 - mse: 0.2518
Epoch 4/150
94/94 ━━━━━━━━━━━━━━━━━━━━ 0s 627us/step - loss: 0.2465 - mse: 0.2465
Epoch 5/150
94/94 ━━━━━━━━━━━━━━━━━━━━ 0s 509us/step - loss: 0.2334 - mse: 0.2334
Epoch 6/150
94/94 ━━━━━━━━━━━━━━━━━━━━ 0s 679us/step - loss: 0.2332 - mse: 0.2332
Epoch 7/150
94/94 ━━━━━━━━━━━━━━━━━━━━ 0s 677us/step - loss: 0.2196 - mse: 0.2196
Epoch 8/150
94/94 ━━━━━━━━━━━━━━━━━━━━ 0s 566us/step - loss: 0.2231 - mse: 0.2231
Epoch 9/150
94/94 ━━━━━━━━━━━━━━━━━━━━ 0s 452us/step - loss: 0.2186 - mse: 0.2186
Epoch 10/150
94/94 ━━━━━━━━━━━━━━━━━━━━ 0s 502us/step - loss: 0.2217 - mse: 0.2217
Epoch 11/150
94/94 ━━━━━━━━━━━━━━━━━━━━ 0s 683us/step - loss: 0.2113 - mse: 0.2113
Epoch 12/150
94/94 ━━━━━━━━━━━━━━━━━━━━ 0s 498us/step - loss: 0.2130 - mse: 0.2130
Epoch 13/150


- Find the RMSE on train data 

In [61]:
# RMSE on train data (scaled)
np.sqrt(mean_squared_error(y_train_scaled, cv_baseline_preds))

0.4428941678214032

- Convert the scaled predictions back to original scale 
- Calculate RMSE in the original units with `y_train` and `baseline_preds` 

In [62]:
# Convert the predictions back to original scale
baseline_preds = ss_y.inverse_transform(cv_baseline_preds)

# RMSE on train data (original scale)
np.sqrt(mean_squared_error(y_train, baseline_preds))

4034.1190263390417

## Intentionally Overfitting a Model

Now that you've developed a baseline model, its time to intentionally overfit a model. To overfit a model, you can:
* Add layers
* Make the layers bigger
* Increase the number of training epochs

Again, be careful here. Think about the limitations of your resources, both in terms of your computers specs and how much time and patience you have to let the process run. Also keep in mind that you will then be regularizing these overfit models, meaning another round of experiments and more time and resources.

In [63]:
# Define a function that returns a compiled Keras model 
def create_bigger_model():
    overfit = models.Sequential()
    overfit.add(layers.Input((n_features, )))
    overfit.add(layers.Dense(32, "relu"))
    overfit.add(layers.Dense(8, "relu"))
    overfit.add(layers.Dense(8, "relu"))
    overfit.add(layers.Dense(4, "relu"))
    overfit.add(layers.Dense(1, "linear"))
    
    overfit.compile(
        optimizer="sgd",
        loss="mse",
        metrics=["mse", "r2_score"]
    )
    
    return overfit

In [64]:
# Wrap the above function for use in cross-validation
keras_wrapper_2 = KerasRegressor(
    create_bigger_model,
    batch_size=256,
    epochs=500
)

In [65]:
# ⏰ This cell may take several mintes to run and you may get a comment from tf about optimizing your machine
# Generate cross-validated predictions
np.random.seed(123)
cv_bigger_model_preds = cross_val_predict(keras_wrapper_2, X_train_all, y_train_scaled)

Epoch 1/500
94/94 ━━━━━━━━━━━━━━━━━━━━ 0s 801us/step - loss: 0.7468 - mse: 0.7468 - r2_score: 0.2598
Epoch 2/500
94/94 ━━━━━━━━━━━━━━━━━━━━ 0s 864us/step - loss: 0.3121 - mse: 0.3121 - r2_score: 0.6909
Epoch 3/500
94/94 ━━━━━━━━━━━━━━━━━━━━ 0s 937us/step - loss: 0.2696 - mse: 0.2696 - r2_score: 0.7269
Epoch 4/500
94/94 ━━━━━━━━━━━━━━━━━━━━ 0s 725us/step - loss: 0.2550 - mse: 0.2550 - r2_score: 0.7444
Epoch 5/500
94/94 ━━━━━━━━━━━━━━━━━━━━ 0s 958us/step - loss: 0.2340 - mse: 0.2340 - r2_score: 0.7620
Epoch 6/500
94/94 ━━━━━━━━━━━━━━━━━━━━ 0s 717us/step - loss: 0.2260 - mse: 0.2260 - r2_score: 0.7743
Epoch 7/500
94/94 ━━━━━━━━━━━━━━━━━━━━ 0s 899us/step - loss: 0.2196 - mse: 0.2196 - r2_score: 0.7749
Epoch 8/500
94/94 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.2190 - mse: 0.2190 - r2_score: 0.7792
Epoch 9/500
94/94 ━━━━━━━━━━━━━━━━━━━━ 0s 870us/step - loss: 0.2165 - mse: 0.2165 - r2_score: 0.7806
Epoch 10/500
94/94 ━━━━━━━━━━━━━━━━━━━━ 0s 926us/step - loss: 0.2151 - mse: 0.2151 - r2_score

In [66]:
# RMSE on train data (scaled)
np.sqrt(mean_squared_error(y_train_scaled, cv_bigger_model_preds))

0.4530844199744746

## Regularizing the Model to Achieve Balance  

Now that you have a powerful model (albeit an overfit one), we can now increase the generalization of the model by using some of the regularization techniques we discussed. Some options you have to try include:  
* Adding dropout
* Adding L1/L2 regularization
* Altering the layer architecture (add or remove layers similar to above)  

This process will be constrained by time and resources. Be sure to test at least two different methodologies, such as dropout and L2 regularization. If you have the time, feel free to continue experimenting. 

In [67]:
# Define a function that returns a compiled Keras model 
def create_regularized_model():
    # Initialize model
    model = models.Sequential()

    # First hidden layer
    model.add(layers.Input((n_features,)))
    model.add(layers.Dense(32, activation='relu'))
    model.add(layers.Dropout(0.4))

    # Second hidden layer
    model.add(layers.Dense(8, activation='relu'))
    model.add(layers.Dropout(0.2))
    
    # Third hidden layer
    model.add(layers.Dense(8, activation='relu'))

    # Output layer
    model.add(layers.Dense(1, activation='linear'))

    # Compile the model
    model.compile(optimizer='SGD',
                  loss='mse',
                  metrics=['mse'])
    
    return model

In [68]:
# Wrap the above function for use in cross-validation
keras_wrapper_3 = KerasRegressor(
    create_regularized_model,
    batch_size=256,
    epochs=150
)

In [69]:
# ⏰ This cell may take several minutes to run, and you may get a comment from tf about optimizing your machine
# Generate cross-validated predictions
np.random.seed(123)
cv_dropout_preds = cross_val_predict(keras_wrapper_3, X_train_all, y_train_scaled)

Epoch 1/150
94/94 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 0.8413 - mse: 0.8413
Epoch 2/150
94/94 ━━━━━━━━━━━━━━━━━━━━ 0s 826us/step - loss: 0.5168 - mse: 0.5168
Epoch 3/150
94/94 ━━━━━━━━━━━━━━━━━━━━ 0s 780us/step - loss: 0.4530 - mse: 0.4530
Epoch 4/150
94/94 ━━━━━━━━━━━━━━━━━━━━ 0s 679us/step - loss: 0.4325 - mse: 0.4325
Epoch 5/150
94/94 ━━━━━━━━━━━━━━━━━━━━ 0s 971us/step - loss: 0.4232 - mse: 0.4232
Epoch 6/150
94/94 ━━━━━━━━━━━━━━━━━━━━ 0s 683us/step - loss: 0.3912 - mse: 0.3912
Epoch 7/150
94/94 ━━━━━━━━━━━━━━━━━━━━ 0s 845us/step - loss: 0.4038 - mse: 0.4038
Epoch 8/150
94/94 ━━━━━━━━━━━━━━━━━━━━ 0s 688us/step - loss: 0.3829 - mse: 0.3829
Epoch 9/150
94/94 ━━━━━━━━━━━━━━━━━━━━ 0s 889us/step - loss: 0.3658 - mse: 0.3658
Epoch 10/150
94/94 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.3544 - mse: 0.3544
Epoch 11/150
94/94 ━━━━━━━━━━━━━━━━━━━━ 0s 848us/step - loss: 0.3418 - mse: 0.3418
Epoch 12/150
94/94 ━━━━━━━━━━━━━━━━━━━━ 0s 895us/step - loss: 0.3258 - mse: 0.3258
Epoch 13/150
94/9

In [70]:
# RMSE on train data (scaled)
np.sqrt(mean_squared_error(y_train_scaled, cv_dropout_preds))

0.5197304337245977

## Final Evaluation

Now that you have selected a network architecture, tested various regularization procedures and tuned hyperparameters via a validation methodology, it is time to evaluate your final model on the test set. Fit the model using all of the training data using the architecture and hyperparameters that were most effective in your experiments above. Afterwards, measure the overall performance on the hold-out test data which has been left untouched (and hasn't leaked any data into the modeling process)! 

In [71]:
# ⏰ This cell may take several mintes to run and you may get a comment from tf about optimizing your machine

In [72]:
# Initialize model
model = models.Sequential()

# First hidden layer
model.add(layers.Input((n_features,)))
model.add(layers.Dense(10, activation='relu'))

# Second hidden layer
model.add(layers.Dense(5, activation='relu'))

# Output layer
model.add(layers.Dense(1, activation='linear'))

# Compile the model
model.compile(optimizer='SGD',
              loss='mse',
              metrics=['mse'])

In [73]:
model.fit(X_train_all, y_train_scaled, batch_size=256, epochs=150)

Epoch 1/150
117/117 ━━━━━━━━━━━━━━━━━━━━ 0s 712us/step - loss: 0.6421 - mse: 0.6421
Epoch 2/150
117/117 ━━━━━━━━━━━━━━━━━━━━ 0s 836us/step - loss: 0.2871 - mse: 0.2871
Epoch 3/150
117/117 ━━━━━━━━━━━━━━━━━━━━ 0s 709us/step - loss: 0.2589 - mse: 0.2589
Epoch 4/150
117/117 ━━━━━━━━━━━━━━━━━━━━ 0s 861us/step - loss: 0.2437 - mse: 0.2437
Epoch 5/150
117/117 ━━━━━━━━━━━━━━━━━━━━ 0s 916us/step - loss: 0.2370 - mse: 0.2370
Epoch 6/150
117/117 ━━━━━━━━━━━━━━━━━━━━ 0s 823us/step - loss: 0.2328 - mse: 0.2328
Epoch 7/150
117/117 ━━━━━━━━━━━━━━━━━━━━ 0s 995us/step - loss: 0.2272 - mse: 0.2272
Epoch 8/150
117/117 ━━━━━━━━━━━━━━━━━━━━ 0s 736us/step - loss: 0.2198 - mse: 0.2198
Epoch 9/150
117/117 ━━━━━━━━━━━━━━━━━━━━ 0s 678us/step - loss: 0.2186 - mse: 0.2186
Epoch 10/150
117/117 ━━━━━━━━━━━━━━━━━━━━ 0s 817us/step - loss: 0.2140 - mse: 0.2140
Epoch 11/150
117/117 ━━━━━━━━━━━━━━━━━━━━ 0s 953us/step - loss: 0.2190 - mse: 0.2190
Epoch 12/150
117/117 ━━━━━━━━━━━━━━━━━━━━ 0s 806us/step - loss: 0.2058 - m

In [75]:
np.sqrt(model.evaluate(X_test_all, y_test_scaled)[0])

399/399 ━━━━━━━━━━━━━━━━━━━━ 0s 503us/step - loss: 0.1751 - mse: 0.1751


0.4233493825856387

## Summary

In this lab, you investigated some data from *The Lending Club* in a complete data science pipeline to build neural networks with good performance. You began with reserving a hold-out set for testing which never was touched during the modeling phase. From there, you implemented a k-fold cross validation methodology in order to assess an initial baseline model and various regularization methods. 